# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore using [SMARTS](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) matching
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules and define a couple of useful functions for visualization:

In [30]:
from copy import deepcopy
import os
import multiprocessing as mp

import gsd.hoomd
import mbuild as mb
import numpy as np
import gsd.pygsd
from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict
from cmeutils.gsd_utils import snap_molecule_cluster
def visualize_qcc_input(qcc_input):
    """
    Visualize a quantum chemical input string (for pyscf) using mbuild.
    
    Parameters
    ----------
    qcc_input : str
        Input string to visualize
    """
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()
    
def from_snapshot(snapshot, scale=1.0):
    """
    Convert a hoomd.data.Snapshot or a gsd.hoomd.Snapshot to an
    mbuild Compound.
    
    Parameters
    ----------
    snapshot : hoomd.data.SnapshotParticleData or gsd.hoomd.Snapshot
        Snapshot from which to build the mbuild Compound.
    scale : float, optional, default 1.0
        Value by which to scale the length values
        
    Returns
    -------
    comp : mb.Compound
    """
    comp = mb.Compound()
    bond_array = snapshot.bonds.group
    n_atoms = snapshot.particles.N

    # There will be a better way to do this once box overhaul merged
    try:
        # gsd
        box = snapshot.configuration.box
        comp.box = mb.box.Box(lengths=box[:3] * scale)
    except AttributeError:
        # hoomd
        box = snapshot.box
        comp.box = mb.box.Box(lengths=np.array([box.Lx,box.Ly,box.Lz]) * scale)

    # to_hoomdsnapshot shifts the coords, this will keep consistent
    shift = np.array(comp.box.lengths)/2
    # Add particles
    for i in range(n_atoms):
        name = snapshot.particles.types[snapshot.particles.typeid[i]]
        xyz = snapshot.particles.position[i] * scale + shift
        charge = snapshot.particles.charge[i]

        atom = mb.Particle(name=name, pos=xyz, charge=charge)
        comp.add(atom, label=str(i))

    # Add bonds
    particle_dict = {idx: p for idx, p in enumerate(comp.particles())}
    for i in range(bond_array.shape[0]):
        atom1 = int(bond_array[i][0])
        atom2 = int(bond_array[i][1])
        comp.add_bond([particle_dict[atom1], particle_dict[atom2]])
    return comp

Here's our starting structure, an atomistic (not coarse-grain or united atom) gsd file with 2 p3ht 15-mers:

In [31]:
with gsd.hoomd.open(name='/Users/jimmy/cmelab/data/ITIC/4700e7da2f41cc6c77e08977f1f8d94c/trajectory.gsd', mode='rb') as f:
    snap = f[0]
    
box = snap.configuration.box[:3]
ref_distance = 3.563594872561358
unwrapped_positions = snap.particles.position + snap.particles.image * box

unwrap_snap = deepcopy(snap)
unwrap_snap.particles.position = unwrapped_positions
unwrap_snap.particles.types = [amber_dict[i].symbol for i in snap.particles.types]
comp = from_snapshot(unwrap_snap, scale=0.1*ref_distance)
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next let's use SMARTS matching to detect our chromophores. This SMARTS string is for (a generalized) p3ht. The `conversion_dict` is a dictionary which converts atom type to element.

Note: The positions/orientations in the gsd file are not optimal, so openbabel has a tough time recognizing them as aromatic -- this is why I am defining the SMARTS by element (`[#6]`) instead of aromatic carbon (`c`) and even so, one chromophore is not detected correctly. If we were running a simulation workflow from scratch, I would recommend using the first frame (before any distortion) for smarts matching and then mapping those indices to the final structure.

In [29]:
#smarts_str = "c1ccccc1"
#aaids=[]
#aaids_pent = chromophores.get_chromo_ids_smiles(snap, "c1sc2ccsc2c1", amber_dict)
comp = from_snapshot(unwrap_snap, scale=0.1*ref_distance)

#aaids_benzene = chromophores.get_chromo_ids_smiles(snap, smarts_str, amber_dict)
#aaids_thiothene = chromophores.get_chromo_ids_smiles(snap, "c1sc(C)cc1", amber_dict)
#aaids.extend(aaids_benzene)
#aaids.extend(aaids_thiothene)
#aaids.extend(aaids_pent)


NameError: name 'unwrap_snap' is not defined

For two 15-mers with each monomer being a chromophore, we expect 30 chromophores. Basically, the smarts matching misses one chromophore. So I have to add it manually. 

The visualization below shows the detected chromophores in pink and the missed one in blue:

In [11]:
missed_inds = np.array([10, 11, 12, 19, 26,32,22,21, 30, 28, 20, 97, 98, 99, 100, 166, 168, 169, 171])
#missed_inds = []
for i,p in enumerate(comp.particles()):
    if i in np.hstack(aaids):
        p.name = "Kr"
    elif i in missed_inds:
        p.name = "Kr"
comp.visualize().show()

aaids.append(missed_inds)

NameError: name 'aaids' is not defined

In [28]:
chromo_ids = np.array([0,1,2,4,6,7,10,11,12,13,19,20,22,23,24,25,27,28,29,30,31,32,91,92,93,94,97,98,99,100,101,102,161,162,163,165,166,168,169,170,171,172,178,179,180,181,177,175,176,174,173,17,15,18,16,14,13])
for i,p in enumerate(comp.particles()):
    if i in chromo_ids:
        p.name = "Kr"
comp.visualize().show()

NameError: name 'comp' is not defined

Next let's make a Chromophore object for each detected chromophore and add them to a list:

In [53]:
master_aaids = []

master_aaids = set(master_aaids)

TypeError: unhashable type: 'numpy.ndarray'

Trying to get homo and lumo for a single molecule of itic below. using functions in execute_qcc.py. it looks like i need to use write_qcc_imp() for that I need the snap, amber_dict and atom ids. 

In [ ]:
/Users/jimmy/cmelab/data/ITIC/4700e7da2f41cc6c77e08977f1f8d94c/trajectory.gsd

In [79]:
g = gsd.pygsd.GSDFile(open('/Users/jimmy/cmelab/data/ITIC/4700e7da2f41cc6c77e08977f1f8d94c/trajectory.gsd', "rb"))
trajectory = gsd.hoomd.HOOMDTrajectory(g)
gsd_length= len(trajectory)
with gsd.hoomd.open(name='/Users/jimmy/cmelab/data/ITIC/4700e7da2f41cc6c77e08977f1f8d94c/trajectory.gsd', mode='rb') as f:
    start_snap = f[1]
    end_snap = f[gsd_length - 1]
    
#ids = start_snap.particles.typeid
ids = np.arange(start_snap.particles.N)
start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance

In [80]:
qcc_input = eqcc.write_qcc_inp(start_snap, ids, amber_dict)

In [81]:
print(qcc_input)

C -11.450420114301867 4.913000837449104 -2.334926051478231; C -10.62080547501964 5.878941312912971 -3.003505153040731; C -9.277354928754992 5.605571523789436 -3.199855250696981; H -11.044782373213 6.84450985539344 -3.5172628587292074; C -8.728589746259875 4.38307072270301 -2.906123561243856; H -8.684419366621203 6.441711202744514 -3.6693185990856527; C -9.641090127729601 3.376813665513069 -2.5584215348766683; C -10.962783548139758 3.608038679245979 -2.017867488245809; H -11.441552850507922 2.889600530747444 -1.3854040330456137; H -12.420362207197375 5.140653387192756 -2.1178468888805746; C -9.03839466264171 2.1130988751688307 -2.3547758286999105; C -7.718425485395617 2.467179075364143 -2.382867259363973; C -7.408912393354601 3.8633468304910963 -2.4948400682018637; O -6.4063870637647575 4.476288572434456 -2.7371210282848715; C -9.614186975263781 0.9633958493509596 -1.9660162156628012; C -10.939828607343859 0.7731282864847486 -2.5138563340710043; C -9.101953241132922 -0.22583602320763418

It looks like I have a qcc input IM not sure the move I used to get "ids" was legit . time will tell

In [82]:
homolumo = eqcc.get_homolumo(qcc_input)

In [83]:
print(homolumo)

[-7.35879844 -7.15381864 -1.13025732 -0.94548734]


the above is
HOMO-1, HOMO, LUMO, LUMO+1 above. 

In [84]:
visualize_qcc_input(qcc_input)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

below i want to get the mom lumos for just that back bone.

In [17]:
g = gsd.pygsd.GSDFile(open('/Users/jimmy/cmelab/data/ITIC/4700e7da2f41cc6c77e08977f1f8d94c/trajectory.gsd', "rb"))
trajectory = gsd.hoomd.HOOMDTrajectory(g)
gsd_length= len(trajectory)
with gsd.hoomd.open(name='/Users/jimmy/cmelab/data/ITIC/4700e7da2f41cc6c77e08977f1f8d94c/trajectory.gsd', mode='rb') as f:
    start_snap = f[1]
    end_snap = f[gsd_length - 1]
    

start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance
qcc_input = eqcc.write_qcc_inp(start_snap, chromo_ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
visualize_qcc_input(qcc_input)
print(homolumo)

/Users/jimmy/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/pyscf/semiempirical/mindo3.py:137: RuntimeWarning: divide by zero encountered in true_divide
  mopac_param.E2/distances_in_AA - gamma)
/Users/jimmy/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/pyscf/scf/hf.py:181: RuntimeWarning: invalid value encountered in double_scalars
  cycle+1, e_tot, e_tot-last_hf_e, norm_gorb, norm_ddm)
/Users/jimmy/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/pyscf/scf/hf.py:185: RuntimeWarning: invalid value encountered in double_scalars
  elif abs(e_tot-last_hf_e) < conv_tol and norm_gorb < conv_tol_grad:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

[-5.30240302 -4.0925731  -3.57417916 -3.57347029]


In [10]:
visualize_qcc_input(qcc_input)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

maybe use the freud cluster modeule to see how they. snap molecule cluster is a functiomn that might return the molecule index of each particle


In [45]:
mols = snap_molecule_cluster("/Users/jimmy/cmelab/data/ITIC/4700e7da2f41cc6c77e08977f1f8d94c/trajectory.gsd")

In [46]:
type(mols)
type(chromo_ids)

numpy.ndarray

In [68]:
print(len(chromo_ids))

57


In [48]:
print(len(mols))

186


below im going to sort out the linear algebra of indexing chromophores given the molecule index and the chomo_ids obtained above. we have 186 atoms per molecule. 

In [24]:
big_gsd = "/Users/jimmy/cmelab/data/ITIC/0769578cc7d05faca991a8089c23e0e0/trajectory.gsd"

In [25]:
gsd_mol_index = snap_molecule_cluster(big_gsd)

In [26]:
k = np.count_nonzero(gsd_mol_index==0)
print(k)

186


the below function takes chromo_ids amd gsd_mol_index and gives a list of atom indeces for each chromophore

In [32]:
master_list = []
sublist = chromo_ids
for i in range(len(np.unique(gsd_mol_index))):         
    master_list.append(sublist)
    sublist = [x + k for x in sublist]

In [72]:
total_index = len(np.unique(gsd_mol_index))*np.count_nonzero(gsd_mol_index==0)
print(total_index)

4092


AttributeError: 'list' object has no attribute 'flatten'

In [53]:
molecules = np.unique(gsd_mol_index)

In [14]:
test_ids = np.concatenate(master_list, axis=0)
print(test_ids)

[   0    1    2 ... 3922 3920 3919]


In [33]:

for i,p in enumerate(comp.particles()):
    if i in test_ids:
        p.name = "Kr"
comp.visualize().show()

NameError: name 'test_ids' is not defined

In [4]:
with gsd.hoomd.open(name='/Users/jimmy/cmelab/data/ITIC/0769578cc7d05faca991a8089c23e0e0/trajectory.gsd', mode='rb') as f:
    snap = f[0]
    
box = snap.configuration.box[:3]
ref_distance = 3.563594872561358
unwrapped_positions = snap.particles.position + snap.particles.image * box

unwrap_snap = deepcopy(snap)
unwrap_snap.particles.position = unwrapped_positions
unwrap_snap.particles.types = [amber_dict[i].symbol for i in snap.particles.types]
comp = from_snapshot(unwrap_snap, scale=0.1*ref_distance)
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [34]:
for i,p in enumerate(comp.particles()):
    if i in test_ids:
        p.name = "Kr"
comp.visualize().show()

NameError: name 'test_ids' is not defined

I have a list called master_list that is a list of arrays of particle ids pictured above. 
below i want to plug this into the whole morphCT workflow for shiz and gigs

In [ ]:
g = gsd.pygsd.GSDFile(open('/Users/jimmy/cmelab/data/ITIC/0769578cc7d05faca991a8089c23e0e0/trajectory.gsd', "rb"))
trajectory = gsd.hoomd.HOOMDTrajectory(g)
gsd_length= len(trajectory)
with gsd.hoomd.open(name='/Users/jimmy/cmelab/data/ITIC/0769578cc7d05faca991a8089c23e0e0/trajectory.gsd', mode='rb') as f:
    start_snap = f[0]
    end_snap = f[gsd_length - 1]
    

aaids = master_list
start_box = start_snap.configuration.box[0:3]
end_box = end_snap.configuration.box[0:3]

# convert to angstrom
ref_distance = 3.563594872561358
start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance
end_snap.particles.position *= ref_distance
end_snap.configuration.box[:3] *= ref_distance

chromo_list = []
for i,aaid in enumerate(aaids):
    chromo_list.append(chromophores.Chromophore(i, end_snap, aaid, "acceptor", amber_dict))    
    
qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, end_snap, amber_dict, d_cut=min(end_box)/2)

outpath = os.path.join(os.getcwd(), "output")
if not os.path.exists(outpath):
    os.makedirs(outpath)
s_filename = os.path.join(outpath, "singles_energies.txt")
d_filename = os.path.join(outpath, "dimer_energies.txt")

data = eqcc.singles_homolumo(chromo_list, s_filename)

dimer_data = eqcc.dimer_homolumo(qcc_pairs, d_filename)

eqcc.set_energyvalues(chromo_list, s_filename, d_filename)

kmc_dir = os.path.join(outpath, "kmc")
if not os.path.exists(kmc_dir):
    os.makedirs(kmc_dir)
    
seed = 42

lifetimes = [1.00e-13, 1.00e-12]
jobs_list = kmc.get_jobslist(lifetimes, n_holes=10, seed=seed)

temp = 300
combined_data = kmc.run_kmc(jobs_list, kmc_dir, chromo_list, end_snap, temp, verbose=1)

with open(os.path.join(kmc_dir, "kmc_00.log"), "r") as f:
    lines = f.readlines()
print(*lines)

kmc_analyze.main(combined_data, temp, chromo_list, end_snap, kmc_dir)

In [35]:
g = gsd.pygsd.GSDFile(open('/Users/jimmy/cmelab/data/ITIC/0769578cc7d05faca991a8089c23e0e0/trajectory.gsd', "rb"))
trajectory = gsd.hoomd.HOOMDTrajectory(g)
gsd_length= len(trajectory)
with gsd.hoomd.open(name='/Users/jimmy/cmelab/data/ITIC/0769578cc7d05faca991a8089c23e0e0/trajectory.gsd', mode='rb') as f:
    start_snap = f[0]
    end_snap = f[gsd_length - 1]
    

aaids = master_list
start_box = start_snap.configuration.box[0:3]
end_box = end_snap.configuration.box[0:3]

# convert to angstrom
ref_distance = 3.563594872561358
start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance
end_snap.particles.position *= ref_distance
end_snap.configuration.box[:3] *= ref_distance


In [36]:
chromo_list = []
for i,aaid in enumerate(aaids):
    chromo_list.append(chromophores.Chromophore(i, end_snap, aaid, "acceptor", amber_dict))    
    
qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, end_snap, amber_dict, d_cut=min(end_box)/2)


In [37]:
outpath = os.path.join(os.getcwd(), "output")
if not os.path.exists(outpath):
    os.makedirs(outpath)
s_filename = os.path.join(outpath, "singles_energies.txt")
d_filename = os.path.join(outpath, "dimer_energies.txt")

In [38]:
data = eqcc.singles_homolumo(chromo_list, s_filename)

In [ ]:
%%time
dimer_data = eqcc.dimer_homolumo(qcc_pairs, d_filename)